In [1]:
import requests 
import json
from pprint import pprint
import os
import pandas as pd
import csv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as sts
import gmaps
from scipy.stats import sem
from scipy.stats import linregress
from config import Q1api_key
from config import Q1_GKey

ModuleNotFoundError: No module named 'config'

In [ ]:
# # Import Merged GDP and Weather data
merge_path = os.path.join("Resources", "Merge_data_2005-2015.csv")
merge_df = pd.read_csv(merge_path)

merge_df.head()

In [ ]:
# # Understand the data
max_Rain = merge_df['Rainfall_mm'].max()
min_Rain = merge_df['Rainfall_mm'].min()
print(f"Maximum temperature in data is {max_Rain}")
print(f"Minimum temperature in data is {min_Rain}")

max_GDP = merge_df['GDP_bd'].max()
min_GDP = merge_df['GDP_bd'].min()
print(f"Maximum Gross Domestic Product in data is {max_GDP}")
print(f"Minimum Gross Domestic Product in data is {min_GDP}")

In [ ]:
# # Plot all the data
# Plot the Rainfall versus GDP data to look for trend/features to investigate
ListofISO34Plot = merge_df['ISO3']
# # Make Scatter plot of Rainfall vs GDP  
plt.figure(figsize=(10,5))
plt.scatter(merge_df["GDP_bd"], merge_df["Rainfall_mm"], marker='o', facecolor='b', edgecolors='purple', alpha = 0.75)
plt.title(f"Average GDP vs Average Annual Precipitation (mm)")
plt.xlabel("Average GDP (bil $)")
plt.ylabel("Precipitation (mm)")
plt.tight_layout()
plt.savefig("output_data/Q2/Fig1x_AvTemp_vs_GDP_AllCountries.png")
plt.show()

In [ ]:
# Crop data to area of interest
zone1_df = merge_df.loc[(merge_df['Rainfall_mm']>=150) & (merge_df['Rainfall_mm']<=250)]
zone1_df

zone2_df = merge_df.loc[(merge_df['Rainfall_mm']>=50) & (merge_df['Rainfall_mm']<=149)]
zone2_df

zone3_df = merge_df.loc[(merge_df['Rainfall_mm']> 0) & (merge_df['Rainfall_mm']<=49)]
zone3_df

# Create a merged (zone1, 2 & 3) dataframe
ZONE_df = merge_df.loc[(merge_df['Rainfall_mm']>=150) & (merge_df['Rainfall_mm']<=250) | (merge_df['Rainfall_mm']>=50) & (merge_df['Rainfall_mm']<=149) | (merge_df['Rainfall_mm']> 0) & (merge_df['Rainfall_mm']<=49)]
ZONE_df = ZONE_df.dropna(how='any')


# # Understand the data
ZoneCountries = ZONE_df['ISO3'].nunique()
print(f"No. of countries after filter {ZoneCountries}")

ZONE_df

In [ ]:
# # Looking for Regressions
# Looking at the trend line for temperature over time (10yrs) for each country and GDP over time for 
# the same country storing these values to compare them.

# FOR COMBINED ZONES OF INTEREST
ListofISO3_Merged = ZONE_df['ISO3']
ZONERegressList = []
ZONERegressDictGDP = {}
ZONERegressDictRain = {}
R_ValuesGDP = {}
R_ValuesRain = {}

# Run for loop to get regression values for GDP for each country
for ISO in ListofISO3_Merged:
    ZONECorrelation_df = ZONE_df.loc[ZONE_df['ISO3']== ISO]
    GDP = ZONECorrelation_df['GDP']
    Rain = ZONECorrelation_df['Rainfall_mm']
    Yrs = ZONECorrelation_df['Year']    

    #  LINEAR REGRESSION MODEL 
    # Run the regression and store each of the returned values
    slope, intercept, rvalue, pvalue, stderr = linregress(Yrs, GDP)
    Zone_regression = [slope, intercept, rvalue, pvalue, stderr]
    Zone_Rvalue = Zone_regression[2]
    ZONERegressDictGDP[ISO] = Zone_Rvalue
R_ValuesGDP = ZONERegressDictGDP

# Run for loop to get regression values for annual average Temperature for each country
for ISO in ListofISO3_Merged:
    ZONECorrelation_df = ZONE_df.loc[ZONE_df['ISO3']== ISO]
    GDP = ZONECorrelation_df['GDP']
    Rain = ZONECorrelation_df['Rainfall_mm']
    Yrs = ZONECorrelation_df['Year']    

    #  LINEAR REGRESSION MODEL 
    # Run the regression and store each of the returned values
    slope, intercept, rvalue, pvalue, stderr = linregress(Yrs, Rain)
    Zone_regression = [slope, intercept, rvalue, pvalue, stderr]
    Zone_Rvalue = Zone_regression[2]
    ZONERegressDictRain[ISO] = Zone_Rvalue
R_ValuesRain = ZONERegressDictRain


In [ ]:
# # Comparing GDP and Temperature regressions
# Looking at the difference between the r-value for the GDP trend line and the annual temperature of a given 
# country would help ascertain any relationship.
# For example, a high, positive r-value for the GDP trend line and a smiliar high, positive r-value for the 
# annual temperature trend line indicates the possibility of a correlation.

# Create a df to look at the GDP and Temperature regressions
RegressList ={"RAIN_REGRESS": R_ValuesRain,"GDP_REGRESS": R_ValuesGDP}
Regress_df = pd.DataFrame(RegressList)
Regress_df = Regress_df.reset_index()
Regress_df = Regress_df.rename(columns={'index': 'ISO3'})
Regress_df['Regression_diff'] = Regress_df['RAIN_REGRESS'] - Regress_df['GDP_REGRESS']
Regress_df = Regress_df.sort_values(by='Regression_diff', ascending=True)
Regress_df.tail(50)

In [ ]:
# # Dual axis bar/line chart
# A bar chart/ line graph to compare GDP and annual temperature trends over time.
# Make a ISO List based on findings from regress data frame
RegressList = ['COL', 'FJI', 'MHL', 'GUY', 'DOM', 'CYP', 'ISL', 'JPN', 'GRC', 'AND', 'SVN', 'CAF', 'ITA', 'LBY', 'IRL', 'FIN']
for ISO in RegressList:
    BarGraph_df = ZONE_df.loc[ZONE_df['ISO3']== ISO]
    yearList = BarGraph_df['Year'].unique()
    yearNo = np.array(range(len(yearList)))
    GDPNos = BarGraph_df['GDP_bd']
    RainNos = BarGraph_df['Rainfall_mm']
    fig, ax = plt.subplots()
    ax2 = ax.twinx()
    ax.bar(yearList, GDPNos, color=(63/255,79/255,232/255,0.7), label='GDP(bill $)')
    ax2.plot(yearList, RainNos, color='red', label='Precipitation(mm)')
    ax.set_xticklabels(yearList)
    ax.legend(loc='center left', bbox_to_anchor=(0, -0.2))
    ax2.legend(loc='center left', bbox_to_anchor=(0.6, -0.2))
    ax.set_title(f"GDP bars and Average Annual Precipitation(mm) Line Graph_{ISO}")
    ax.set_ylabel('GDP (bil,$)')
    ax2.set_ylabel('Precipitation(mm)')
    ax.set_xlabel('Time (Yr.)')
    plt.tight_layout()
    plt.savefig(f"output_data/Q2/Fig2_GDP_BarChart_RainLineChart_{ISO}_2005-2015.png")
